In [29]:
!pip show resampy

Name: resampy
Version: 0.4.3
Summary: Efficient signal resampling
Home-page: https://github.com/bmcfee/resampy
Author: Brian McFee
Author-email: brian.mcfee@nyu.edu
License: ISC
Location: /usr/local/lib/python3.11/dist-packages
Requires: numba, numpy
Required-by: 


In [30]:
!pip install pyrubberband

In [31]:
!apt-get install -y rubberband-cli


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
rubberband-cli is already the newest version (2.0.0-2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [32]:
!pip install resampy

In [33]:
# 📁 preprocess.ipynb - Updated with Delta + Delta-Delta Features

## 1. Import Libraries

import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib
from tqdm import tqdm



In [34]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
os.getcwd()

'/content'

In [36]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

# print("Path to dataset files:", path)

In [37]:
# import os
# import shutil

# # Source: Kaggle dataset location (read-only)
# kaggle_dataset_path = "/kaggle/input/ravdess-emotional-speech-audio"

# # Destination: Your project folder in Google Drive
# target_path = "/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/raw/ravdess"

# # Make sure destination exists
# os.makedirs(target_path, exist_ok=True)

# # Loop over actor folders and copy them
# for folder in os.listdir(kaggle_dataset_path):
#     src = os.path.join(kaggle_dataset_path, folder)
#     dest = os.path.join(target_path, folder)
#     if os.path.isdir(src):
#         shutil.copytree(src, dest, dirs_exist_ok=True)
#         print(f"✅ Copied: {folder}")

# print("🎉 All Actor folders copied to:", target_path)


In [38]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/raw/ravdess")

['Actor_12',
 'Actor_16',
 'Actor_17',
 'Actor_04',
 'Actor_07',
 'Actor_13',
 'Actor_06',
 'Actor_15',
 'Actor_08',
 'Actor_05',
 'Actor_09',
 'Actor_01',
 'Actor_11',
 'Actor_10',
 'Actor_02',
 'Actor_23',
 'Actor_18',
 'Actor_14',
 'Actor_19',
 'Actor_21',
 'Actor_03',
 'Actor_20',
 'Actor_22',
 'Actor_24']

In [39]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition')

# You're telling Python:
# “Treat /content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/ as if it’s the root of my codebase.
# So anything inside that — like utils/, models/, src/ — can now be imported easily.”


In [40]:
os.getcwd()

'/content'

In [41]:
from src.utils.augment import apply_augmentation_pipeline

In [42]:


# ## 2. Feature Extraction Function (Log-Mel + Delta + Delta-Delta)

# def extract_logmel(audio, sr, n_mels=128, max_len=173):
#     mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
#     logmel = librosa.power_to_db(mel, ref=np.max)

#     # Delta and Delta-Delta
#     delta = librosa.feature.delta(logmel)
#     delta2 = librosa.feature.delta(logmel, order=2)

#     # Stack them as 3 channels: [static, delta, delta-delta]
#     logmel_stack = np.stack([logmel, delta, delta2], axis=0)  # (3, n_mels, time)
#     logmel_stack = librosa.util.fix_length(logmel_stack, size=max_len, axis=2)

#     # Normalize each channel independently
#     for i in range(3):
#         _min = logmel_stack[i].min()
#         _max = logmel_stack[i].max()
#         logmel_stack[i] = (logmel_stack[i] - _min) / (_max - _min + 1e-6)

#     return logmel_stack  # shape: (3, 128, time)


In [43]:
!ls "/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/raw/"

ravdess


In [44]:
!ls "/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/raw/ravdess"


Actor_01  Actor_04  Actor_07  Actor_10	Actor_13  Actor_16  Actor_19  Actor_22
Actor_02  Actor_05  Actor_08  Actor_11	Actor_14  Actor_17  Actor_20  Actor_23
Actor_03  Actor_06  Actor_09  Actor_12	Actor_15  Actor_18  Actor_21  Actor_24


In [45]:


## 3. Load & Preprocess RAVDESS Audio Dataset
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/raw/ravdess/"

os.listdir(DATA_PATH)
# emotion_dict = {'neutral': 0, 'calm': 1, 'happy': 2, 'sad': 3, 'angry': 4, 'fearful': 5, 'disgust': 6, 'surprised': 7}


['Actor_12',
 'Actor_16',
 'Actor_17',
 'Actor_04',
 'Actor_07',
 'Actor_13',
 'Actor_06',
 'Actor_15',
 'Actor_08',
 'Actor_05',
 'Actor_09',
 'Actor_01',
 'Actor_11',
 'Actor_10',
 'Actor_02',
 'Actor_23',
 'Actor_18',
 'Actor_14',
 'Actor_19',
 'Actor_21',
 'Actor_03',
 'Actor_20',
 'Actor_22',
 'Actor_24']

In [46]:
# Extract emotion label from RAVDESS filename
def extract_emotion(filename):
    emotion_map = {
        '01': 'neutral',
        '02': 'calm',
        '03': 'happy',
        '04': 'sad',
        '05': 'angry',
        '06': 'fearful',
        '07': 'disgust',
        '08': 'surprised'
    }
    parts = filename.split('-')
    if len(parts) > 2:
        emotion_id = parts[2]
        return emotion_map.get(emotion_id, None)
    return None


In [47]:
# Scan dataset and get list of (path, label)
file_label_pairs = []
for root, _, files in os.walk(DATA_PATH):
    for file in files:
        if file.endswith('.wav'):
            emotion = extract_emotion(file)
            if emotion:
                file_label_pairs.append((os.path.join(root, file), emotion))

print(f"Total files with valid emotion: {len(file_label_pairs)}")


Total files with valid emotion: 1440


In [48]:
# Encode string labels into integers
labels = [label for _, label in file_label_pairs]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# Save label encoder
joblib.dump(label_encoder, 'label_encoder.joblib')

['label_encoder.joblib']

In [49]:
X = []
y = []

# Iterate through each actor's folder
for actor_folder in tqdm(sorted(os.listdir(DATA_PATH)), desc="Processing Actors"):
    actor_path = os.path.join(DATA_PATH, actor_folder)

    if not os.path.isdir(actor_path):
        continue

    # Process each audio file in the actor folder
    for file in os.listdir(actor_path):
        if not file.endswith(".wav"):
            continue

        audio_path = os.path.join(actor_path, file)

        try:
            features = apply_augmentation_pipeline(audio_path)
            if features is not None:
                X.append(features)
                y.append(extract_emotion(file))
        except Exception as e:
            print(f"Error processing {file}: {e}")


Processing Actors: 100%|██████████| 24/24 [02:55<00:00,  7.32s/it]


In [50]:

X = np.array(X)  # shape: (samples, 3, 128, 173)
y = np.array(y)
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (1440, 3, 128, 173)
y shape: (1440,)


In [51]:
# ✅ Final Input Shape (for Conv1D):

# (samples, time_steps=173, features=128 × 3 = 384)
# So let’s flatten the channel and mel axis (3 × 128 = 384) and keep time (173) as sequence steps.

In [52]:
# X_reshaped = X.transpose(0, 3, 1, 2).reshape(X.shape[0], 173, 384)
# print("Reshaped X shape:", X_reshaped.shape)
# # Explanation:

# # .transpose(0, 3, 1, 2) → makes shape (samples, time=173, channels=3, mel=128)

# # Then reshape into (samples, 173, 384) to merge channels * mel_bins per timestep.

In [53]:
# X = X_reshaped
# print("Final X shape:", X.shape)

In [54]:

## 6. Save Processed Data

np.save("/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/features/features_v6.npy", X)
np.save("/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/features/labels_v6.npy", y)



In [55]:
!ls "/content/drive/MyDrive/Colab Notebooks/voice_emotion_recognition/data/features/"

features.npy	 features_v5.npy  labels.npy	 labels_v5.npy
features_v4.npy  features_v6.npy  labels_v4.npy  labels_v6.npy
